# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [50]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [51]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,1.21,86,40,14.92,CL,1712547117
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,5.48,82,40,5.00,RU,1712547347
2,2,grytviken,-54.2811,-36.5092,2.18,71,2,2.53,GS,1712547310
3,3,udachny,66.4167,112.4000,-12.38,67,9,2.25,RU,1712547171
4,4,port-aux-francais,-49.3500,70.2167,5.53,62,75,8.02,TF,1712547336


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [52]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 800,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_cities_df = city_data_df[
    (city_data_df['Cloudiness'] == 0) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5)
]

# Drop any rows with null values
reduced_cities_df.dropna()

# Display sample data
reduced_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,sao joao da barra,-21.6403,-41.0511,23.65,81,0,2.58,BR,1712547496
251,251,acapulco de juarez,16.8634,-99.8901,25.05,88,0,2.57,MX,1712547109
291,291,rampur,28.8167,79.0333,24.43,8,0,3.12,IN,1712547585
376,376,sultanah,24.4926,39.5857,23.40,21,0,2.06,SA,1712547599
399,399,saint-pierre,-21.3393,55.4781,24.97,65,0,3.09,RE,1712547233
432,432,brisas de zicatela,15.8369,-97.0419,26.96,73,0,3.08,MX,1712547396
496,496,kidal,18.4411,1.4078,26.61,6,0,3.51,ML,1712547620
537,537,porbandar,21.6422,69.6093,26.14,78,0,3.60,IN,1712547372
544,544,dalby,-27.1833,151.2667,26.29,36,0,3.08,AU,1712547398
560,560,santiago,-33.4569,-70.6483,22.70,44,0,2.57,CL,1712547278


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
31,sao joao da barra,BR,-21.6403,-41.0511,81,
251,acapulco de juarez,MX,16.8634,-99.8901,88,
291,rampur,IN,28.8167,79.0333,8,
376,sultanah,SA,24.4926,39.5857,21,
399,saint-pierre,RE,-21.3393,55.4781,65,
432,brisas de zicatela,MX,15.8369,-97.0419,73,
496,kidal,ML,18.4411,1.4078,6,
537,porbandar,IN,21.6422,69.6093,78,
544,dalby,AU,-27.1833,151.2667,36,
560,santiago,CL,-33.4569,-70.6483,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "limit":1,
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sao joao da barra - nearest hotel: Pousada Mediterrâneo
acapulco de juarez - nearest hotel: Hotel del Valle
rampur - nearest hotel: تمنا ہوٹل اور ریستوران
sultanah - nearest hotel: فندق جلنار
saint-pierre - nearest hotel: Tropic Hotel
brisas de zicatela - nearest hotel: Casa de Olas
kidal - nearest hotel: No hotel found
porbandar - nearest hotel: Toran Tourist Bungalow
dalby - nearest hotel: No hotel found
santiago - nearest hotel: apart arturo prat


,City,Country,Lat,Lng,Humidity,Hotel Name
31,sao joao da barra,BR,-21.6403,-41.0511,81,Pousada Mediterrâneo
251,acapulco de juarez,MX,16.8634,-99.8901,88,Hotel del Valle
291,rampur,IN,28.8167,79.0333,8,تمنا ہوٹل اور ریستوران
376,sultanah,SA,24.4926,39.5857,21,فندق جلنار
399,saint-pierre,RE,-21.3393,55.4781,65,Tropic Hotel
432,brisas de zicatela,MX,15.8369,-97.0419,73,Casa de Olas
496,kidal,ML,18.4411,1.4078,6,No hotel found
537,porbandar,IN,21.6422,69.6093,78,Toran Tourist Bungalow
544,dalby,AU,-27.1833,151.2667,36,No hotel found
560,santiago,CL,-33.4569,-70.6483,44,apart arturo prat


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 800,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ['Country', 'Hotel Name']
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)